Author: Sebastiano Busato
Date: 01/03/2023
What this code does: read each file, remove missing rows, do some other fun stuff, then merge 

Dependencies below

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


First dataset: read the training trait data 

In [ ]:
df1 = pd.read_csv("/home/sbusato/G2F/Data/Data_for_merge/Original/1_Training_Trait_Data_2014_2021_MODSB.csv")


In [ ]:
df1.columns

In [ ]:
#pull out desired columns
df1_filt = df1.loc[:,['Env', 'Year', 'Field_Location', 'Hybrid_orig_name', 'Plot_Area_ha', 'plantdate_parsed','harvestdate_parsed','Yield_Mg_ha']]
#remove rows with missing yield 
df1_filt = df1_filt[~df1_filt['Yield_Mg_ha'].isnull()]


In [ ]:

# are there remaining NAs?
df1_filt[df1_filt.isnull().any(axis=1)] #yes
df1_filt.isnull().sum() # only in date harvested


In [ ]:

df1_dates = df1_filt.loc[:,['plantdate_parsed','harvestdate_parsed']]
df1_dates['harvestdate_parsed'] = pd.to_datetime(df1_dates['harvestdate_parsed'], errors='coerce',yearfirst=True)
df1_dates['plantdate_parsed'] = pd.to_datetime(df1_dates['plantdate_parsed'], errors='coerce',yearfirst=True)
df1_dates['days_growth'] = df1_dates['harvestdate_parsed']-df1_dates['plantdate_parsed']

In [ ]:
df1_dates['days_growth'].mean() #mean growth is 159 days

In [ ]:
#pull out values of plantdate where harvestdate is missing
a = df1_filt.loc[df1_filt['harvestdate_parsed'].isnull(),['plantdate_parsed']]
#convert to datetime and add 159 days
b = pd.to_datetime(a['plantdate_parsed'], errors='coerce',yearfirst=True)+pd.to_timedelta('159 days')
#replace missing values with calculations
df1_filt.loc[df1_filt['harvestdate_parsed'].isnull(),['harvestdate_parsed']] = b

df1_filt.isnull().sum()
#no more missing! 

Step 2: Import and modify metadata 

In [ ]:
df2 = pd.read_csv("/home/sbusato/G2F/Data/Data_for_merge/Original/2_Training_Meta_Data_2014_2021.csv", encoding='latin-1')
df2_filt = df2.loc[:,['Env', 'Year', 'Experiment_Code', 'Treatment', 'City', 'Farm', 'Field',
       'Soil_Taxonomic_ID and horizon description, if known',
       'Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)',
       'Weather_Station_Latitude (in decimal numbers NOT DMS)',
       'Weather_Station_Longitude (in decimal numbers NOT DMS)',
       'Date_weather_station_placed', 'Date_weather_station_removed',
       'Previous_Crop', 'Pre-plant_tillage_method(s)',
       'In-season_tillage_method(s)',
       'Type_of_planter (fluted cone; belt cone; air planter)',
       'System_Determining_Moisture', 'Pounds_Needed_Soil_Moisture',
       'Latitude_of_Field_Corner_#1 (lower left)',
       'Longitude_of_Field_Corner_#1 (lower left)',
       'Latitude_of_Field_Corner_#2 (lower right)',
       'Longitude_of_Field_Corner_#2 (lower right)',
       'Latitude_of_Field_Corner_#3 (upper right)',
       'Longitude_of_Field_Corner_#3 (upper right)',
       'Latitude_of_Field_Corner_#4 (upper left)',
       'Longitude_of_Field_Corner_#4 (upper left)']]

df2_filt.isnull().sum() # Treatment is categorical, unlikely to be important
                        # City, Farm, Filed - same
                        # Soil taxID - too many missing
                        # Long & Lat - Only use one - fewer missing in Weather station location 


In [ ]:

#df2_filt['Treatment'].unique()

df2_b = df2.loc[:,['Latitude_of_Field_Corner_#1 (lower left)',
       'Longitude_of_Field_Corner_#1 (lower left)','Weather_Station_Latitude (in decimal numbers NOT DMS)',
       'Weather_Station_Longitude (in decimal numbers NOT DMS)',]]

df2_b
df2_b['latitude_diff'] = df2_b['Latitude_of_Field_Corner_#1 (lower left)'] - df2_b['Weather_Station_Latitude (in decimal numbers NOT DMS)']
df2_b['longitude_diff'] = df2_b['Longitude_of_Field_Corner_#1 (lower left)'] - df2_b['Weather_Station_Longitude (in decimal numbers NOT DMS)']

df2_b.mean()

plt.hist(df2_b.loc[~df2_b['latitude_diff'].isnull(),'latitude_diff'])
plt.hist(df2_b.loc[~df2_b['longitude_diff'].isnull(),'longitude_diff']) # one has a large difference! (7 decimals - 600 or so km)



In [ ]:
df2_b.loc[df2_b['Latitude_of_Field_Corner_#1 (lower left)'].isna(),['Latitude_of_Field_Corner_#1 (lower left)']] = df2_b[df2_b['Latitude_of_Field_Corner_#1 (lower left)'].isna()]['Weather_Station_Latitude (in decimal numbers NOT DMS)']
df2_b.loc[df2_b['Longitude_of_Field_Corner_#1 (lower left)'].isna(),['Longitude_of_Field_Corner_#1 (lower left)']] = df2_b.loc[df2_b['Longitude_of_Field_Corner_#1 (lower left)'].isna()]['Weather_Station_Longitude (in decimal numbers NOT DMS)']
df2_b.isna().sum() #only 16 nan remaining


In [ ]:
df2_filt[df2_filt['Latitude_of_Field_Corner_#1 (lower left)'].isna()]

In [ ]:
df2_filt = df2.loc[:,['Env', 'Year', 
       'Weather_Station_Latitude (in decimal numbers NOT DMS)',
       'Weather_Station_Longitude (in decimal numbers NOT DMS)',
       'Latitude_of_Field_Corner_#1 (lower left)',
       'Longitude_of_Field_Corner_#1 (lower left)',]]

df2_filt.columns = (['Env','Year','WS_Long','WS_Lat','Field_Long','Field_Lat'])

df2_filt.loc[df2_filt['Field_Long'].isna(),['Field_Long']] = df2_filt.loc[df2_filt['Field_Long'].isna()]['WS_Long']
df2_filt.loc[df2_filt['Field_Lat'].isna(),['Field_Lat']] = df2_filt.loc[df2_filt['Field_Lat'].isna()]['WS_Lat']

df2_filt = df2_filt.loc[:,['Env','Year','Field_Long','Field_Lat']]
df2_filt

In [ ]:
#plt.hist(df2_filt['Field_Lat'])
df2_filt[df2_filt['Field_Lat']>0]


Load soil data and filter

In [ ]:
df3 = pd.read_csv("/home/sbusato/G2F/Data/Data_for_merge/Original/3_Training_Soil_Data_2015_2021.csv")
len(df3) # 141 soil samples
df3.isnull().sum() #date received has 17 null
#these are missing from almost all 
#BpH                           130
#Zinc ppm Zn                   138
#Iron ppm Fe                   138
#Manganese ppm Mn              138
#Copper ppm Cu                 138
#Boron ppm B

In [ ]:
df3[df3['Date Received'].isna()].isnull().sum()  #these have almost no info
df3_filt = df3[~df3['Date Received'].isna()]

df3_filt.isna().sum()

df3_filt.columns # some need dropped 
#WDRF Buffer pH -- we have 1:1 soil pH for all 
#comments - w/e
#BpH, Zinc, Iron, Manganese, Copper, Boron - missing for almost all
#Date received and reported - irrelevant


df3_filt = df3_filt.loc[:,['Year', 'Env', 'E Depth',
       '1:1 Soil pH', '1:1 S Salts mmho/cm', 'Texture No',
       'Organic Matter LOI %', 'Nitrate-N ppm N', 'lbs N/A', 'Potassium ppm K',
       'Sulfate-S ppm S', 'Calcium ppm Ca', 'Magnesium ppm Mg',
       'Sodium ppm Na', 'CEC/Sum of Cations me/100g', '%H Sat', '%K Sat',
       '%Ca Sat', '%Mg Sat', '%Na Sat', 'Mehlich P-III ppm P', '% Sand',
       '% Silt', '% Clay', 'Texture']]

df3_filt.isna().sum() # few remaining 


In [ ]:
df3_filt[df3_filt.isnull().any(axis=1)] #only 6 samples have all the NaN, they're only two locations
#df3_filt.to_csv("3_soil_pruned.csv") #manually interpolated in excel

df3_filt = pd.read_csv("/home/sbusato/G2F/Data/Data_for_merge/Original/3_Soil_ManInterp_noNA.csv")
df3_filt.isna().sum() # All NAs are gone

#df3_filt.loc[:,'Texture'] = pd.Categorical(df3_filt.loc[:,'Texture']).codes #replace texture with category
#df3_filt


Read and prune preprocessed weather data 

In [ ]:
df4 = pd.read_csv("/home/sbusato/G2F/Data/Data_for_merge/Original/4_weather_summarized_by_week_noNAs.csv")
df4.isna().sum() # No NAs
df4_filt = df4

Read genotype data

In [ ]:
df5 = pd.read_csv("/home/sbusato/G2F/Data/Data_for_merge/Original/5_Geno_LD_filt_noNAs_01062023.csv")
 # No NAs - great 

In [ ]:
df5.isna().sum().sum()

Read EC data

In [ ]:
df6 = pd.read_csv("/home/sbusato/G2F/Data/Data_for_merge/Original/6_Training_EC_Data_2014_2021.csv")
df6.isna().sum().sum() # No NAs - great 
df6_filt = df6

Merge everything

In [ ]:
df12 = pd.merge(df1_filt,df2_filt, on=["Env","Year"])

In [ ]:
df123 = pd.merge(df12,df3_filt, on=["Env","Year"])

In [ ]:
df1234 = pd.merge(df123,df4_filt, on=["Env","Year"])

In [ ]:
df1234 = df1234.rename(columns = {'Hybrid_orig_name':'Hybrid'})
df5 = df5.rename(columns = {'Unnamed: 0':'Hybrid'})

In [ ]:
df12345 = pd.merge(df1234,df5, on=['Hybrid'])

In [ ]:
df123456 = pd.merge(df12345,df6_filt, on=["Env"])

In [ ]:
df123456.to_csv("all_df_merge_01062023.csv")

In [ ]:
a = df123456.isna().sum() 
a[a>0]
